In [104]:
# -------
# IMPORT LIBRAIRIES
# -------
import numpy as np
import pandas as pd
import plotly.express as px
from python_module import sabr
from python_module import blackscholes
from sklearn.linear_model import LinearRegression
from scipy.cluster.hierarchy import linkage, fcluster

# -------
# DISPLAY SETTINGS
# -------
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.options.display.float_format = lambda x: f'{x:_.4f}'

In [120]:
# -------
# GENERATE MARKET DATA & PORTFOLIO
# -------
S = F = 100
alpha = 0.2
beta = 1
rho = -0.4
nu = 0.5
market_data_list = list()
for time_to_maturity in np.linspace(start=1, stop=250, num=10, dtype=int):
    T = time_to_maturity / 250
    for K in np.linspace(start=80, stop=100, num=10):
        IV = sabr.compute_vol(F, K, T, alpha, beta, rho, nu)
        pricing_results = blackscholes.compute_option(S, K, T, 0, IV, 'put', True)
        market_data_list.append({'T': time_to_maturity, 'K': K, 'IV': IV, **pricing_results})
market_data_df = pd.DataFrame(market_data_list)
market_data_df['weights'] = abs(np.random.normal(size=market_data_df.shape[0]))
market_data_df['abs_theta'] = abs(market_data_df['theta'])

In [121]:
market_data_df.tail()

,T,K,IV,price,delta,gamma,vega,theta,vanna,volga,weights,abs_theta
95,250,91.1111,0.2117,4.3751,-0.2927,0.0162,0.3438,-0.0144,-0.5419,29.5622,0.4625,0.0144
96,250,93.3333,0.2088,5.1372,-0.3318,0.0174,0.3630,-0.0150,-0.3929,17.0855,0.4899,0.0150
97,250,95.5556,0.2060,5.9948,-0.3731,0.0184,0.3786,-0.0155,-0.2161,6.9943,0.8103,0.0155
98,250,97.7778,0.2035,6.9522,-0.4160,0.0192,0.3901,-0.0157,-0.0166,0.3529,0.6056,0.0157
99,250,100.0000,0.2012,8.0119,-0.4599,0.0197,0.3969,-0.0158,0.1985,-1.9962,0.3016,0.0158


In [122]:
maturities = market_data_df['T'].unique()
for maturity in maturities:
    maturity_index = market_data_df[market_data_df['T']==maturity].index
    temp_df = market_data_df.loc[maturity_index]
    X = temp_df[['gamma', 'vanna', 'volga']].to_numpy()
    Y = temp_df['theta']
    model = LinearRegression(fit_intercept=False)
    model.fit(X, Y)
    market_data_df.loc[maturity_index, 'theta_pred'] = model.predict(X)
    market_data_df.loc[maturity_index, 'cost_of_gamma'] = temp_df[['gamma', 'vanna', 'volga']].multiply(model.coef_)['gamma'].to_numpy() * -1
    market_data_df.loc[maturity_index, 'cost_of_vanna'] = temp_df[['gamma', 'vanna', 'volga']].multiply(model.coef_)['vanna'].to_numpy() * -1
    market_data_df.loc[maturity_index, 'cost_of_volga'] = temp_df[['gamma', 'vanna', 'volga']].multiply(model.coef_)['volga'].to_numpy() * -1

In [123]:
market_data_df.tail()

,T,K,IV,price,delta,gamma,vega,theta,vanna,volga,weights,abs_theta,theta_pred,cost_of_gamma,cost_of_vanna,cost_of_volga
95,250,91.1111,0.2117,4.3751,-0.2927,0.0162,0.3438,-0.0144,-0.5419,29.5622,0.4625,0.0144,-0.0144,0.0133,0.0008,0.0003
96,250,93.3333,0.2088,5.1372,-0.3318,0.0174,0.3630,-0.0150,-0.3929,17.0855,0.4899,0.0150,-0.0150,0.0143,0.0006,0.0002
97,250,95.5556,0.2060,5.9948,-0.3731,0.0184,0.3786,-0.0155,-0.2161,6.9943,0.8103,0.0155,-0.0155,0.0151,0.0003,0.0001
98,250,97.7778,0.2035,6.9522,-0.4160,0.0192,0.3901,-0.0157,-0.0166,0.3529,0.6056,0.0157,-0.0157,0.0157,0.0000,0.0000
99,250,100.0000,0.2012,8.0119,-0.4599,0.0197,0.3969,-0.0158,0.1985,-1.9962,0.3016,0.0158,-0.0158,0.0162,-0.0003,-0.0000


In [134]:
X = market_data_df[['cost_of_gamma', 'cost_of_vanna', 'cost_of_volga']].to_numpy()
Z = linkage(X, method='ward', metric='euclidean')
num_clusters = 10
clusters = fcluster(Z, num_clusters, criterion='maxclust')
market_data_df['cluster'] = [str(x) for x in clusters]

In [135]:
px.scatter(data_frame=market_data_df, x='K', y='T', color='cluster', size='gamma')